In [5]:
import pandas as pd

In [2]:
df = pd.read_csv('animelists_cleaned.csv/animelists_cleaned.csv')

In [3]:
df.head()

,username,anime_id,my_watched_episodes,my_start_date,my_finish_date,my_score,my_status,my_rewatching,my_rewatching_ep,my_last_updated,my_tags
0,karthiga,21,586,0000-00-00,0000-00-00,9,1,NaN,0,2013-03-03 10:52:53,NaN
1,karthiga,59,26,0000-00-00,0000-00-00,7,2,NaN,0,2013-03-10 13:54:51,NaN
2,karthiga,74,26,0000-00-00,0000-00-00,7,2,NaN,0,2013-04-27 16:43:35,NaN
3,karthiga,120,26,0000-00-00,0000-00-00,7,2,NaN,0,2013-03-03 10:53:57,NaN
4,karthiga,178,26,0000-00-00,0000-00-00,7,2,0.0,0,2013-03-27 15:59:13,NaN


In [4]:
df.shape

(31284030, 11)

In [6]:
import operator

In [149]:
def calculate_order_greedy(Item_list, rating, user_id):
    pie = {}
    rank = {}
    for i in Item_list:
        print("item is: ", i)
        pie[i] = preference_func(i, i+1, user_id, rating) - preference_func(i+1, i, user_id, rating)
    while (len(Item_list) > 0):
        t = max(pie.items(), key = operator.itemgetter(1))[0]
        rank[t] = len(Item_list)
        Item_list.remove(t)
        for k in Item_list:
            pie[k] = pie[k] + preference_func(t,k, user_id, rating) - preference_func(k,t, user_id, rating)
    return rank
        
        

In [6]:
rating = pd.read_csv('rating.csv/rating.csv')

In [122]:
mask =(rating['user_id'].value_counts() >= 10) & (rating['user_id'].value_counts() <= 15)


In [123]:
users = mask[mask == True].index

In [124]:
rating_2 = rating[rating['user_id'].isin(users)]

In [125]:
rating_2.shape

(63270, 3)

In [126]:
rating_2['anime_id'].nunique()

3275

In [78]:
rating.shape

(6337241, 3)

In [16]:
rating = rating[rating['rating'] >0]


In [48]:
rating[rating['user_id'] == 3]

,user_id,anime_id,rating
156,3,20,8
157,3,154,6
158,3,170,9
159,3,199,10
160,3,225,9
161,3,341,6
162,3,430,7
163,3,527,7
164,3,552,7
165,3,813,10


In [99]:
def indicator_f(x):
    if(x <0):
        r = 1
    else:
        r = 0
    return r

def Kendall_CC(u_id, v_id, df):
    S = 0
    
    user_anm_rating = pd.merge(df[df['user_id'] == u_id], df[df['user_id'] == v_id], on='anime_id', how='inner', \
                               suffixes={'_U', '_V'})
    if(len(user_anm_rating)>10):
        val = 0
        for a in range(len(user_anm_rating)-1):
            e = (user_anm_rating.loc[a,'rating_U'] - user_anm_rating.loc[a+1,'rating_U'])*(user_anm_rating.loc[a,'rating_V'] - user_anm_rating.loc[a+1,'rating_V'])
            val = val + indicator_f(e)
            #print(val)
        S = 1-4*(val/len(user_anm_rating)*(len(user_anm_rating)-1))
    return S
        
        
    

In [32]:
user_anm_rating= pd.merge(rating[rating['user_id'] == 3], rating[rating['user_id'] == 5], on='anime_id', how='inner', suffixes={'_U', '_V'}) 

In [151]:
def preference_func(A, B, user_id, rating):
    neighbs = Neighb(user_id, rating) ## return dictionary
    num = 0.0
    denom = 0.0
    for key, val in neighbs.items():
        prod = rating.loc[((rating['user_id'] == key)&(rating['anime_id'] == A)), 'rating'] - rating.loc[((rating['user_id'] == key)&(rating['anime_id'] == B)), 'rating']
        num = num + prod*val
        denom = denom+val
    print("preference func done for: ", A," & ", B)
    return num/denom

In [50]:
###preference_func(1535,20,3 )
import numpy as np

In [152]:
def Neighb(user_id, rating, cutoff = 10):
    all_users = list((rating['user_id']).unique())#.remove(user_id)
    all_users.remove(user_id)
    neighb_score = {}
    for u in all_users:
        print("user no: ", u)
        neighb_score[u] = Kendall_CC(user_id, u, rating)
    top_10 = sorted(neighb_score.items(), key=operator.itemgetter(1), reverse=True)[:cutoff]
    return dict(top_10)
        
    

In [105]:
##Neighb(3, rating_2)

In [62]:
user_anm_rating= pd.merge(rating[rating['user_id'] == 3], rating[rating['user_id'] == 9], on='anime_id', how='inner', suffixes={'_U', '_V'}) 

In [142]:
d = {'a':10, 'b':12, 'c':1, 'd':0.77}
d = dict(sorted(d.items(), key=operator.itemgetter(1), reverse=True)[:3])

In [127]:
all_items = rating_2['anime_id'].unique()

In [146]:
len(all_items)

3275

In [145]:
all_ranks = calculate_order_greedy(all_items, rating_2, 28)

KeyboardInterrupt: 

In [147]:
## Stopped due to long time

In [148]:
rating_2['user_id'].nunique()

5094

In [14]:
ratings_3 = rating[rating['user_id'].isin([23,79,1000,45,20,10,1,560,999,5000,2000,5706])]

In [15]:
ratings_3['user_id'].value_counts()

5000    441
5706    334
1       153
2000    113
79      108
23       72
1000     41
45       26
20       19
999      16
560      12
10        8
Name: user_id, dtype: int64

In [ ]:
all_animes = ratings_3['anime_id']

### Reco Engine 

In [184]:
def calculate_order_greedy(rating, user_id):
    pie = {}
    rank = {}
    my_neighbours, Item_list = Neighb(user_id, rating)
    #print("Item list:", Item_list)
    #print('\n')
    #print((Item_list)[:10])
    #Item_list = Item_list[:10]
    for i in Item_list:
        #print("my I is:", i)
        second_list = Item_list.copy()
        second_list.remove(i)
        for j in second_list:
        #print("Item is: ", Item_list[i], ":",Item_list[i+1])
            pie[i] = preference_func(i, j, user_id, rating,my_neighbours) - preference_func(j, i, user_id, rating,my_neighbours)
        #print("my pie:", pie, '\n' )
    #print("Pie is: ",pie)
    while (len(Item_list) > 0):
        t = max(pie.items(), key = operator.itemgetter(1))[0]
        #print("T is:", t)
        rank[t] = len(Item_list)
        Item_list.remove(t)
        pie.pop(t, None)
        #print("Item list updated is:", Item_list)
        for k in Item_list:
            #print("\nPrinting k as:", k)
            pie[k]= pie[k] + preference_func(t,k, user_id, rating,my_neighbours) - preference_func(k,t, user_id, rating,my_neighbours)
        #print("Pie updates as:", pie)
    return rank


def Neighb(user_id, rating, cutoff = 10):
    all_users = list((rating['user_id']).unique())#.remove(user_id)
    all_users.remove(user_id)
    neighb_score = {}
    Item_list = []
    for u in all_users:
        print("user is", u)
        neighb_score[u] = Kendall_CC(user_id, u, rating)
        Item_list = Item_list + list(rating.loc[rating['user_id'] == u, 'anime_id'])
#         print("length of item_list for user:", len(Item_list))
#         print("\n Item_list is", Item_list)
    #print("original lenght of items", len(Item_list))
    top_10 = dict(sorted(neighb_score.items(), key=operator.itemgetter(1), reverse=True)[:10])
    print(top_10)
    return top_10, list(set(Item_list))

def preference_func(A, B, user_id, rating, my_neighbs):
    #neighbs = Neighb(user_id, rating) ## return dictionary
    pf = 0.00
    num = 0.0
    denom = 0.0
    for key, val in my_neighbs.items():
        #print("val for:", key, "is", val)
        if(val!= 0.00):
            #print("key is", key, "Anime:", A, "&", B)
            mask1 = sum((rating['user_id'] == key)&(rating['anime_id'] == A))
            #print( "Mask 1", mask1)
            mask2 = sum((rating['user_id'] == key)&(rating['anime_id'] == B))
            #print( "Mask 2", mask2)
            if ((mask1 >0 )&( mask2 >0)):
                prod = rating.loc[((rating['user_id'] == key)&(rating['anime_id'] == A)), 'rating'].item() - rating.loc[((rating['user_id'] == key)&(rating['anime_id'] == B)), 'rating'].item()     
                #print("product for rating ", A," & ", B, " by ", key, " is:", prod)
                num = num + prod*val
                denom = denom+val
    if(denom > 0):
        pf = num/denom
        #print("Done preference as:", num/denom)
    return pf


def indicator_f(x):
    if(x <0):
        r = 1
    else:
        r = 0
    return r

def Kendall_CC(u_id, v_id, df):
    S = 0
    user_anm_rating = pd.merge(df[df['user_id'] == u_id], df[df['user_id'] == v_id], on='anime_id', how='inner', \
                               suffixes={'_U', '_V'})
    #print("\nshape of common animes rated:", user_anm_rating.shape)
    if(len(user_anm_rating)>5):
        val = 0
        for a in range(len(user_anm_rating)-1):
            e = (user_anm_rating.loc[a,'rating_U'] - user_anm_rating.loc[a+1,'rating_U'])*(user_anm_rating.loc[a,'rating_V'] - user_anm_rating.loc[a+1,'rating_V'])
            val = val + indicator_f(e)
            #print(val)
        S = 1-4*(val/(len(user_anm_rating)*(len(user_anm_rating)-1)))
    return S

In [178]:
#import operator

my_rank = calculate_order_greedy(ratings_3, 5000)

user is 1
user is 10
user is 20
user is 23
user is 45
user is 79
user is 560
user is 999
user is 1000
user is 2000
user is 5706
{20: 1.0, 5706: 0.9910696952048146, 2000: 0.9885714285714285, 79: 0.9819004524886877, 1000: 0.9637681159420289, 999: 0.9636363636363636, 23: 0.9215686274509804, 560: 0.8571428571428572, 45: 0.5714285714285714, 1: 0}


[1, 5, 6, 22535, 4106, 15, 16, 19, 20, 6164]
my I is: 1
my I is: 5
my I is: 6
my I is: 22535
my I is: 4106
my I is: 15
my I is: 16
my I is: 19
my I is: 20
my I is: 6164
Pie is:  {1: 0.0, 5: 0.0, 6: 0.0, 22535: 0.0, 4106: 0.0, 15: 0.0, 16: 0.0, 19: 0.0, 20: 0.0, 6164: 0.0}
T is: 1
Item list updated is: [5, 6, 22535, 4106, 15, 16, 19, 20, 6164]

Printing k as: 5

Printing k as: 6

Printing k as: 22535

Printing k as: 4106

Printing k as: 15

Printing k as: 16

Printing k as: 19

Printing k as: 20

Printing k as: 6164
Pie updates as: {5: -0.9649225715626466, 6: 0.0, 22535: -4.0, 4106: 0.0, 15: 2.0, 16: -2.0, 19: -4.0, 20: 0.0, 6164: 0.0}
T is: 15
It

In [179]:
my_rank

{1: 10, 15: 9, 20: 8, 6164: 7, 6: 6, 4106: 5, 16: 4, 5: 3, 22535: 2, 19: 1}

In [181]:
dict(sorted(my_rank.items(), key=operator.itemgetter(1), reverse=True))

{1: 10, 15: 9, 20: 8, 6164: 7, 6: 6, 4106: 5, 16: 4, 5: 3, 22535: 2, 19: 1}

### checking performance

In [185]:
import time
start_time = time.time()
my_rank = calculate_order_greedy(ratings_3, 5000)
elapsed_time = time.time() - start_time
print("time elapsed for one user and all possible items:", elapsed_time)

user is 1
user is 10
user is 20
user is 23
user is 45
user is 79
user is 560
user is 999
user is 1000
user is 2000
user is 5706
{20: 1.0, 5706: 0.9910696952048146, 2000: 0.9885714285714285, 79: 0.9819004524886877, 1000: 0.9637681159420289, 999: 0.9636363636363636, 23: 0.9215686274509804, 560: 0.8571428571428572, 45: 0.5714285714285714, 1: 0}
time elapsed for one user and all possible items: 9761.602122068405


In [186]:
my_rank

{22535: 554,
 16512: 553,
 19315: 552,
 16592: 551,
 2476: 550,
 5: 549,
 225: 548,
 9790: 547,
 859: 546,
 860: 545,
 3785: 544,
 8407: 543,
 298: 542,
 6164: 541,
 591: 540,
 886: 539,
 1749: 538,
 60: 537,
 232: 536,
 10321: 535,
 430: 534,
 193: 533,
 330: 532,
 15583: 531,
 1696: 530,
 15451: 529,
 11617: 528,
 13807: 527,
 8074: 526,
 8425: 525,
 243: 524,
 12711: 523,
 15085: 522,
 426: 521,
 6682: 520,
 5391: 519,
 13759: 518,
 1639: 517,
 14813: 516,
 9741: 515,
 28623: 514,
 48: 513,
 10330: 512,
 2156: 511,
 4214: 510,
 168: 509,
 12471: 508,
 10444: 507,
 10491: 506,
 257: 505,
 8560: 504,
 4726: 503,
 32: 502,
 690: 501,
 11005: 500,
 857: 499,
 9201: 498,
 7334: 497,
 5337: 496,
 11633: 495,
 7593: 494,
 7705: 493,
 22199: 492,
 3791: 491,
 5958: 490,
 3588: 489,
 6880: 488,
 4981: 487,
 1138: 486,
 11385: 485,
 1157: 484,
 3270: 483,
 3366: 482,
 1409: 481,
 1410: 480,
 10015: 479,
 1887: 478,
 1132: 477,
 6702: 476,
 790: 475,
 6033: 474,
 12581: 473,
 6500: 472,
 10620